In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm

[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
task_name = 'word_sv_agreement_vp'
task = TASKS[task_name](get_task_params(), data_dir='../data')
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)

Vocab len:  10032


In [3]:
config={'student_exp_name':'lisa_fd131',
    'teacher_exp_name':'0.001_samira_offlineteacher_v11',
    'teacher_config':'small_lstm_v4',
    'task_name':'word_sv_agreement_vp',
    'student_model':'cl_lstm',
    'teacher_model':'cl_lstm',
    'student_config':'small_lstm_v4',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
       }

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
get_teacher_model(config, task, tchr_hparams, cl_token)


model config: small_lstm_v4
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_samira_offlineteacher_v11_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_lisa_fd131
No checkpoint found ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_samira_offlineteacher_v11_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_lisa_fd131
model config: small_lstm_v4
No checkpoint found ../tf_ckpts/word_sv_agreement_vp/cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_samira_offlineteacher_v11


(<tf2_models.lm_lstm.ClassifierLSTM at 0x7f23600da050>,
 <tensorflow.python.training.tracking.util.Checkpoint at 0x7f2360120ed0>)

In [ ]:
teacher_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())
train = teacher_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = teacher_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = teacher_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [ ]:
train = student_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = student_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = student_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [11]:
student_models = []
student_labels = []

import logging
tf.get_logger().setLevel(logging.ERROR)

In [12]:
config={'student_exp_name':'gc_f_std124',
    'teacher_exp_name':'gc_o_tchr124',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_lstm',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_lstm_v4',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }

# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2lstm_124')

config={'student_exp_name':'gc_f_std125',
    'teacher_exp_name':'gc_o_tchr125',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_lstm',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_lstm_v4',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }

# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2lstm_125')

config={'student_exp_name':'gc_f_std130',
    'teacher_exp_name':'gc_o_tchr130',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_lstm',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_lstm_v4',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }


# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2lstm_130')

config={'student_exp_name':'gc_f_std131',
    'teacher_exp_name':'gc_o_tchr131',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_lstm',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_lstm_v4',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }

# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2lstm_131')

model config: small_lstm_v4
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr124_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_f_std124
Restored student from ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr124_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_f_std124/ckpt-60
model config: small_lstm_v4
Restored student from ../tf_ckpts/word_sv_agreement_vp/cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr124/ckpt-60
model config: small_lstm_v4
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr125_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_

In [13]:

for lbl, mdl in zip(student_labels, student_models):
    train = mdl.evaluate(task.train_dataset, steps=task.n_train_batches)
    valid = mdl.evaluate(task.valid_dataset, steps=task.n_valid_batches)
    test = mdl.evaluate(task.test_dataset, steps=task.n_test_batches)

    print(lbl)
    
    print("train:", train)
    print("valid:", valid)
    print("test:", test)

 1380/21933 [>.............................] - ETA: 13:49 - loss: 0.0893 - classification_loss: 0.0750 - sparse_categorical_accuracy: 0.9712

KeyboardInterrupt: 

In [ ]:
# student_models = []
# student_labels = []

config={'student_exp_name':'gc_f_std148',
    'teacher_exp_name':'gc_o_tchr148',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2_shared',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2ugpt_148')

config={'student_exp_name':'gc_f_std149',
    'teacher_exp_name':'gc_o_tchr149',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2_shared',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2ugpt_149')

config={'student_exp_name':'gc_f_std150',
    'teacher_exp_name':'gc_o_tchr150',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2_shared',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')


std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2ugpt_150')

config={'student_exp_name':'gc_f_std151',
    'teacher_exp_name':'gc_o_tchr151',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2_shared',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2ugpt_151')

In [ ]:
for lbl, mdl in zip(student_labels, student_models):
    train = mdl.evaluate(task.train_dataset, steps=task.n_train_batches)
    valid = mdl.evaluate(task.valid_dataset, steps=task.n_valid_batches)
    test = mdl.evaluate(task.test_dataset, steps=task.n_test_batches)

    
    print(lbl)
    print("train:", train)
    print("valid:", valid)
    print("test:", test)

In [15]:
student_models = []
student_labels = []

config={'student_exp_name':'gc_f_std169',
    'teacher_exp_name':'gc_o_tchr169',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_bert',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2bert_169')

config={'student_exp_name':'gc_f_std170',
    'teacher_exp_name':'gc_o_tchr170',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_bert',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2bert_170')

config={'student_exp_name':'gc_f_std171',
    'teacher_exp_name':'gc_o_tchr171',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_bert',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2bert_171')

config={'student_exp_name':'gc_f_std172',
    'teacher_exp_name':'gc_o_tchr172',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_bert',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2bert_172')

model config: small_gpt_v9
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr169_student_cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_gc_f_std169
Restored student from ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr169_student_cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_gc_f_std169/ckpt-60
model config: small_lstm_v4
Restored student from ../tf_ckpts/word_sv_agreement_vp/cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr169/ckpt-60
model config: small_gpt_v9
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr170_student_cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt

In [10]:
for lbl, mdl in zip(student_labels, student_models):
    train = mdl.evaluate(task.train_dataset, steps=task.n_train_batches)
    valid = mdl.evaluate(task.valid_dataset, steps=task.n_valid_batches)
    test = mdl.evaluate(task.test_dataset, steps=task.n_test_batches)

    print(lbl)
    print("train:", train)
    print("valid:", valid)
    print("test:", test)

21933/21933 [==============================] - 197s 9ms/step - loss: 0.1454 - classification_loss: 0.1339 - sparse_categorical_accuracy: 0.9573
lstm2bert_172
train: [0.07114030811792034, 0.05962038, 0.9771142]
valid: [0.14247776621695213, 0.13095777, 0.95731705]
test: [0.1453856613380595, 0.13386622, 0.9573224]


In [7]:
# student_models = []
# student_labels = []

config={'student_exp_name':'gc_f_std165',
    'teacher_exp_name':'gc_o_tchr165',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2gpt_165')

config={'student_exp_name':'gc_f_std166',
    'teacher_exp_name':'gc_o_tchr166',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
config['distill_mode'] = 'online'
config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')
std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2gpt_166')

config={'student_exp_name':'gc_f_std167',
    'teacher_exp_name':'gc_o_tchr167',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')
std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2gpt_167')

config={'student_exp_name':'gc_f_std168',
    'teacher_exp_name':'gc_o_tchr168',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_lstm',
    'student_model':'cl_gpt2',
    'teacher_config':'small_lstm_v4',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
# config['distill_mode'] = 'online'
# config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')
std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('lstm2gpt_168')

model config: small_gpt_v9
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr165_student_cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_gc_f_std165
Restored student from ../tf_ckpts/word_sv_agreement_vp/offline_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr165_student_cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_gc_f_std165/ckpt-60
model config: small_lstm_v4
Restored student from ../tf_ckpts/word_sv_agreement_vp/cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr165/ckpt-60
model config: small_gpt_v9
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/online_pure_dstl_4_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_gc_o_tchr166_student_cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_

In [ ]:
for lbl, mdl in zip(student_labels, student_models):
    train = mdl.evaluate(task.train_dataset, steps=task.n_train_batches)
    valid = mdl.evaluate(task.valid_dataset, steps=task.n_valid_batches)
    test = mdl.evaluate(task.test_dataset, steps=task.n_test_batches)

    print(lbl)
    print("train:", train)
    print("valid:", valid)
    print("test:", test)

In [ ]:
# student_models = []
# student_labels = []

config={'student_exp_name':'gc_f_std116',
    'teacher_exp_name':'gc_o_tchr116',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_bert',
    'student_model':'cl_bert_shared',
    'teacher_config':'small_gpt_v9',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
config['distill_mode'] = 'online'
config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('bert2ubert_116')

config={'student_exp_name':'gc_f_std117',
    'teacher_exp_name':'gc_o_tchr117',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_bert',
    'student_model':'cl_bert_shared',
    'teacher_config':'small_gpt_v9',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
config['distill_mode'] = 'online'
config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')
std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('bert2ubert_117')

config={'student_exp_name':'gc_f_std118',
    'teacher_exp_name':'gc_o_tchr118',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_bert',
    'student_model':'cl_bert_shared',
    'teacher_config':'small_gpt_v9',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
config['distill_mode'] = 'online'
config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')
std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('bert2ubert_118')

config={'student_exp_name':'gc_f_std119',
    'teacher_exp_name':'gc_o_tchr119',
    'task_name':'word_sv_agreement_vp',
    'teacher_model':'cl_bert',
    'student_model':'cl_bert_shared',
    'teacher_config':'small_gpt_v9',
    'student_config':'small_gpt_v9',
    'distill_config':'pure_dstl_4_crs_slw',
    'distill_mode':'offline',
    'chkpt_dir':'../tf_ckpts',
     }
config['distill_mode'] = 'online'
config['student_exp_name'] = config['student_exp_name'].replace('_f_', '_o_')
std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True

student_model, ckpt = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
teacher_model = get_teacher_model(config, task, tchr_hparams, cl_token)

student_models.append(student_model)
student_labels.append('bert2ubert_119')

In [ ]:
for lbl, mdl in zip(student_labels, student_models):
    train = mdl.evaluate(task.train_dataset, steps=task.n_train_batches)
    valid = mdl.evaluate(task.valid_dataset, steps=task.n_valid_batches)
    test = mdl.evaluate(task.test_dataset, steps=task.n_test_batches)

    print(lbl)
    print("train:", train)
    print("valid:", valid)
    print("test:", test)

In [16]:
import tensorflow_probability as tfp

def test_for_calibration(model, task, n_bins=10):
    preds = []
    correct_class_probs = []
    predicted_class_probs = []
    pred_logits = []
    y_trues = []
    batch_count = task.n_valid_batches
    for x, y in task.valid_dataset:
        y = tf.cast(y, tf.int32)
        logits = model(x)
        pred_logits.extend(logits.numpy())
        pred = tf.argmax(logits, axis=-1)
        prob = task.get_probs_fn()(logits, labels=y, temperature=1)
        preds.extend(pred.numpy())
        y_trues.extend(y.numpy())
        batch_indexes = tf.cast(tf.range(len(y), dtype=tf.int32), dtype=tf.int32)
        true_indexes = tf.concat([batch_indexes[:,None], y[:,None]], axis=1)
        pred_indexes = tf.concat([batch_indexes[:,None], tf.cast(pred[:,None], tf.int32)], axis=1)

        correct_class_probs.extend(tf.gather_nd(prob, true_indexes).numpy())
        predicted_class_probs.extend(tf.gather_nd(prob, pred_indexes).numpy())

        batch_count -= 1
        if batch_count == 0:
            break

    model_accuracy = np.asarray(preds) == np.asarray(y_trues)

    return model_accuracy, predicted_class_probs, correct_class_probs, pred_logits, y_trues


for label,model in zip(student_labels, student_models):
    print(label)
    print(model.model_name)

    model_accuracy, predicted_class_probs, correct_class_probs, model_logits, model_trues= test_for_calibration(model, task, n_bins=20)

    #print(len(model_accuracy))
    #print(len(predicted_class_probs))
    ##plot_calibration(model_accuracy, predicted_class_probs, correct_class_probs, n_bins=20)
    #  plt.show()
    model_ece = tfp.stats.expected_calibration_error(
        1000000,
        logits=model_logits,
        labels_true=model_trues,
    )
    print(model_ece.numpy())
    
    print("#######################")

lstm2bert_169
cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2
0.054943
#######################
lstm2bert_170
cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2
0.056423344
#######################
lstm2bert_171
cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2
0.053588104
#######################
lstm2bert_172
cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2
0.05076441
#######################


In [31]:
from tqdm import tqdm



def evaluate_vp_cl(model, verb_infl, noun_infl, task, split='test', batch_size=1000, cls=False):
    distance_hits = Counter()
    distance_total = Counter()
    diff_hits = Counter()
    diff_total = Counter()

    test_data = task.databuilder.as_dataset(split=split, batch_size=batch_size)
    e = 0
    for examples in test_data:
        e += 1
        print(e, end="\r")
        sentences = examples['sentence']
        #bos = tf.cast(task.databuilder.sentence_encoder().encode(constants.bos) * tf.ones((sentences.shape[0],1)), dtype=tf.int64)
        eos = tf.cast(task.databuilder.sentence_encoder().encode(constants.eos) *tf.ones((sentences.shape[0],1)), dtype=tf.int64)

        sentences = tf.concat([sentences, eos], axis=-1)

        verb_position = examples['verb_position']+int(cls)  #+1 because of adding bos.
        # The verb it self is also masked
        mask = tf.cast(tf.sequence_mask(verb_position,maxlen=tf.shape(sentences)[1]), dtype=tf.int64)
        max_length = tf.reduce_max(verb_position + 1)

        last_index_mask = tf.gather(tf.eye(tf.shape(sentences)[1], dtype=tf.int64),verb_position)
        last_index_mask = last_index_mask * eos[0]

        inputs = (sentences * mask + last_index_mask)[:,:max_length]

#         print(sentences[0])
#         print(task.databuilder.sentence_encoder().decode(inputs[0]))
#         break
        s_shape = tf.shape(inputs)
        batch_size, length = s_shape[0], s_shape[1]
        verb_classes = examples['verb_class']
        actual_verbs = examples['verb']
        #inflected_verbs = [verb_infl[v.decode("utf-8")] for v in actual_verbs.numpy()]

        distances = examples['distance'].numpy()
        nz = examples['n_intervening'].numpy()
        n_diffs = examples['n_diff_intervening'].numpy()

        actual_verb_indexes = [task.databuilder.sentence_encoder().encode(v)[0] for v in actual_verbs.numpy()]
        #inflected_verb_indexes = [task.databuilder.sentence_encoder().encode(v)[0] for v in inflected_verbs]


        predictions = model(inputs, training=False)
        predictions = np.argmax(predictions, axis=-1)
        corrects = predictions == verb_classes

        for i, c in enumerate(corrects):
            if actual_verb_indexes[i] == 10035 or actual_verb_indexes[i] == 2:
                continue
            if nz[i] > 4 or distances[i] > 16:
                continue

            distance_total[distances[i]] += 1
            distance_hits[distances[i]] += int(c)
            if nz[i] == n_diffs[i]:
                n = nz[i]
                diff_total[n] += 1
                diff_hits[n] += int(c)
    
    return distance_hits, distance_total, diff_hits, diff_total


infl_eng = inflect.engine()
verb_infl, noun_infl = gen_inflect_from_vocab(infl_eng, 'wiki.vocab')

for label,model in zip(student_labels, student_models):
    print(label)
    print(model.model_name)
    print('##################################')
    distance_hits, distance_total, diff_hits, diff_total = evaluate_vp_cl(model, verb_infl, noun_infl, task)
    compute_and_print_acc_stats(distance_hits, distance_total, diff_hits, diff_total)
    break

lstm2bert_169
cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2
##################################
Accuracy by distance
1 | 0.96 901360
2 | 0.97 82669
3 | 0.96 70115
4 | 0.95 105620
5 | 0.94 57954
6 | 0.93 36349
7 | 0.92 27752
8 | 0.91 20455
9 | 0.90 14638
10 | 0.89 11078
11 | 0.88 8586
12 | 0.88 6232
13 | 0.86 4811
14 | 0.85 3482
15 | 0.86 2563
16 | 0.84 1935
Micro accuracy (distance): 0.9565918829978481
Macro accuracy (distance): 0.9067219673893061
Accuracy by intervenings:
0 | 0.96 1100963
1 | 0.84 50969
2 | 0.75 8949
3 | 0.68 1810
4 | 0.60 457
Micro accuracy (intervenings): 0.9563512123994539
Macro accuracy (intervenings): 0.7655287274231238
